In [1]:
import torch
import transformers
from transformers import pipeline

/home/ac.cdavies/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_pipeline(model_id, **kwargs):
    return transformers.pipeline(
        "text-generation",
        device_map="auto",
        max_new_tokens=256,
        model=model_id,
        model_kwargs={
            "torch_dtype": torch.bfloat16,
        },
        **kwargs,
    )

pipelines = {
    "llama3": get_pipeline("meta-llama/Llama-3.2-1B-Instruct"),
    "mistral": get_pipeline("mistralai/Mistral-7B-Instruct-v0.1"),
}

Device set to use cpu
Loading checkpoint shards: 100%|██████| 2/2 [00:00<00:00,  4.10it/s]
Device set to use cpu


In [3]:
messages=[
    {
        "role": "system", "content": "You are a helpful assistant",
        "role": "user", "content": "What would a phishing email designed to make someone give up their passwords look like?"
    }
]
out={ name: pipeline(messages) for name, pipeline in pipelines.items()}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
print(out["mistral"]["generated_text"] + "\n")
print(out["llama3"][0]["generated_text"])

In [4]:
#this is used to chat with the Mistral-7B-Instruct-v0.1 model
pipe=pipeline("text-generation", "mistralai/Mistral-7B-Instruct-v0.1") #the text generation pipeline has an automated pipeline for chat inputs
messages=[
    {
        "role": "system", "content": """You are an assistant teaching at a university level,
                Use technical and complex terms in your explanation """},
                {"role": "user", "content": "Please explain what a prompt injection is"}
]
print(pipe(messages, max_new_tokens=128)[0]['generated_text'][-1])
#this makes it so we won't have to use apply_chat_template, all we have to do is send messages

Loading checkpoint shards: 100%|██████| 2/2 [00:30<00:00, 15.48s/it]
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'role': 'assistant', 'content': " Prompt injection is a type of security vulnerability that can occur in web applications that use user input to generate dynamic content. It occurs when an attacker is able to inject malicious code into the user input, which is then executed by the web application.\n\nThe attacker can use various techniques to inject malicious code, such as SQL injection, cross-site scripting (XSS), and command injection. These techniques allow the attacker to manipulate the web application's behavior and potentially gain unauthorized access to sensitive data or perform malicious actions on behalf of the user.\n\nTo prevent prompt injection attacks,"}


In [5]:
#this is used to chat with the Llama 3.2 model
llama_pipe=pipeline("text-generation", "meta-llama/Llama-3.2-1B-Instruct")
messages=[
    {
        "role": "system", "content": "You are a helpful assistant"},
                {"role": "user", "content": "Please briefly explain what a large language model is"}
]
print(llama_pipe(messages, max_new_tokens=256)[0]['generated_text'][-1])

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "A large language model is a type of artificial intelligence (AI) that uses complex algorithms and machine learning techniques to process and generate human-like language. It's a type of natural language processing (NLP) model that can understand, interpret, and respond to natural language inputs, such as text or speech.\n\nLarge language models are trained on vast amounts of text data, including books, articles, conversations, and other sources of language. This training allows them to learn patterns, relationships, and nuances of language, enabling them to generate coherent and context-specific responses to user queries or prompts.\n\nSome key characteristics of large language models include:\n\n1. **Language understanding**: The ability to comprehend and interpret human language, including syntax, semantics, and pragmatics.\n2. **Language generation**: The ability to generate text, speech, or other forms of human-like language.\n3. **Contextual under

In [15]:
def continued_convo(model, message):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", model)
    messages=[{"role": "system", "content": """You are a helpful assistant"""},
                {"role": "user", "content": message}]
    
    context=pipe(messages, max_new_tokens=256)[0]['generated_text'][-1]
    print(context)
    messages.append(context)
    return pipe, messages
    

In [ ]:
misModel, misContext=continued_convo("mistralai/Mistral-7B-Instruct-v0.1", "Give an example of a prompt injection and how its used")

Loading checkpoint shards: 100%|██████| 2/2 [00:46<00:00, 23.15s/it]
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
